## Day 12 - Broken Hot Springs!

**Part 1: How many valid arrangements?**

ugh, combinatorics

e.g.
`.??..??...?##. 1,1,3` has *four* as first two `??` can be `#. #.` | `#. .#` | `.# #.` | `.# .#` - but fifth `?` has to be `#` to form the 3 group

In [114]:
with open("./example.txt") as f:
    example_lines = [line.strip() for line in f.readlines()]

with open("./input.txt") as f:
    input_lines = [line.strip() for line in f.readlines()]

In [115]:
example_lines

['???.### 1,1,3',
 '.??..??...?##. 1,1,3',
 '?#?#?#?#?#?#?#? 1,3,1,6',
 '????.#...#... 4,1,1',
 '????.######..#####. 1,6,5',
 '?###???????? 3,2,1']

In [116]:
import re

def find_ranges_of_Qs(row: str):
    """
    row of the form e.g.
        .??..??...?##.
    
    produce a list of the ranges where there are ?s
    """
    hmm_locs = []
    tmp = (None, None)
    for loc in (s.start() for s in re.finditer(re.escape('?'), row)):
        if None in tmp:
            tmp = (loc, loc)
        elif loc - tmp[1] == 1:
            tmp = (tmp[0], loc)
        else:
            hmm_locs.append(tmp)
            tmp = (loc, loc)
    hmm_locs.append(tmp)

    return hmm_locs

find_ranges_of_Qs(".??..??.?.#???..?.#.#.##..?")

[(1, 2), (5, 6), (8, 8), (11, 13), (16, 16), (26, 26)]

In [117]:
def find_general_ranges(row: str):
    hmm_locs = []
    tmp = (None, None)
    for loc in (s.start() for s in re.finditer(re.escape('?')+"|"+re.escape("#"), row)):
        if None in tmp:
            tmp = (loc, loc)
        elif loc - tmp[1] == 1:
            tmp = (tmp[0], loc)
        else:
            hmm_locs.append(tmp)
            tmp = (loc, loc)
    hmm_locs.append(tmp)

    return hmm_locs

find_general_ranges(".??..??.?.#???..?.#.#.##..?")

[(1, 2),
 (5, 6),
 (8, 8),
 (10, 13),
 (16, 16),
 (18, 18),
 (20, 20),
 (22, 23),
 (26, 26)]

In [118]:
# TEST_STRING = ".??..??.?.#???..?.#.#.##..?"
TEST_STRING = "???.###.?.##"

q = find_ranges_of_Qs(TEST_STRING)
g = find_general_ranges(TEST_STRING)

def map_general_ranges_to_Q_ranges(g: list[tuple[int, int]], q: list[tuple[int, int]]) -> dict:
    q = set(q)
    mappings = {}

    for general_range in g:
        if general_range in q:
            q.remove(general_range)
            mappings[general_range] = general_range
        else:
            # see if there's a sub range?
            for q_range in q:
                qs, qe = q_range
                gs, ge = general_range
                if gs <= qs and qe <= ge:
                    mappings[general_range] = q_range
                    q.remove(q_range)
                    break
            else:
                mappings[general_range] = "PRE_SET"

    return mappings

map_general_ranges_to_Q_ranges(g, q)

{(0, 2): (0, 2), (4, 6): 'PRE_SET', (8, 8): (8, 8), (10, 11): 'PRE_SET'}

In [359]:
from math import comb
from itertools import combinations, product, permutations
from copy import deepcopy

def part1(lines: list[str]) -> int:
    combinations_total = 0

    for line in lines:
        string, rules = line.split()
        rules = tuple(map(int, rules.split(",")))

        general_ranges = find_general_ranges(string)
        Q_ranges = find_ranges_of_Qs(string)

        mapped_general_to_Q = map_general_ranges_to_Q_ranges(general_ranges, Q_ranges)

        # RULES ALIGN WITH SLOTS
        if len(rules) == len(mapped_general_to_Q.keys()):
            # We have 1:1 mapping and can do nCr stuff?
            for rule, (key,val) in zip(rules, mapped_general_to_Q.items()):
                if val == "PRE_SET":
                    # all #s
                    continue
                if key == val:
                    # all ?s
                    s, e = val  # start end of range we can put inside
                    positions = e - s + 1
                    combinations_total += comb(positions, rule)
                else:
                    # so we have the mixed case.
                    g_substring = string[key[0]:key[1]+1]

                    required_substring = rule*"#"
                    amount_needed = rule - g_substring.count("#")

                    q_positions = [i for i in range(val[0], val[1]+1)]

                    combos_to_try = combinations(q_positions, r=amount_needed)

                    if len(tuple(combos_to_try)) > 1:
                        for combo in combos_to_try:
                            test_string = ""
                            for ch_idx, ch in enumerate(g_substring):
                                test_string += ch if ch_idx+key[0] not in combo else "#"
                            if required_substring in test_string:
                                combinations_total += 1
        
        # RULES DO NOT ALIGN WITH SLOTS
        else:
            assert len(rules) > len(mapped_general_to_Q.keys()), f"more keys than rules errrr - {string, rules}"

            all_g_ranges_generator = (
                key for key in mapped_general_to_Q.keys()
            )

            sub_rules = []
            using_range = False
            for rule_idx, rule in enumerate(rules):
                # print("rule", rule)
                if not using_range:
                    using_range = True
                    tmp_range = next(all_g_ranges_generator)
                    tmp_total_pos = tmp_range[1] - tmp_range[0] + 1
                    if rule < tmp_total_pos:
                        sub_rules.append(rule)
                        tmp_total_pos -= rule
                else:
                    if rule < tmp_total_pos and rule_idx != len(rules) - 1:
                        sub_rules.append(rule)
                        tmp_total_pos -= rule
                    else:
                        if rule_idx == len(rules) - 1:
                            rule_string = "#"*rule
                            if rule_string not in "".join([ch for ch in string if ch != "."][-rule:]):
                                sub_rules.append(rule)
                                # if it's already satisfied don't have to worry!

                        # print("dealing with it?", sub_rules, tmp_range)
                        # can't assign to sub rules to this range any longer!
                        g_substring = string[tmp_range[0]:tmp_range[1]+1]

                        # print("g_substring: ", g_substring)
                        # print("sub_rules ", sub_rules)

                        # now we have like e.g. sub rules [2,2] and g_substring ????#??

                        potential_ranges_to_alter = []
                        for sub_rule in sub_rules:
                            # print("sub")
                            alterations = []
                            for i in (range(length := len(g_substring))):
                                # god I love walruses
                                if (tmp_end := i + sub_rule - 1) < length:
                                    alterations.append(
                                        (i, tmp_end)
                                    )
                            potential_ranges_to_alter.append(
                                alterations
                            )
                        
                        # print("potential ranges to alter: ", potential_ranges_to_alter)

                        # now we have a list of lists, where the sub lists have 
                        # tuples of ranges we can edit
                        double_counts_potential_edits = [
                            p for p in product(*potential_ranges_to_alter)
                            if len(p) == len(set(p))
                        ]

                        already_had_any_order = set()
                        potential_edits_reduced = []
                        for p in double_counts_potential_edits:
                            if p not in already_had_any_order:
                                potential_edits_reduced.append(p)
                            for item in permutations(p, r=len(p)):
                                already_had_any_order.add(item)

                        potential_edits = []
                        for not_necessarily_legit_edit in potential_edits_reduced:
                            def do_overlap(t1: tuple[int, int], t2: tuple[int, int]):
                                s1, e1 = t1
                                s2, e2 = t2
                                return s1 <= e2 and e1 >= s2

                            adding = True
                            for t1, t2 in combinations(not_necessarily_legit_edit, r=2):
                                if do_overlap(t1, t2):
                                    adding = False
                                    break
                            if adding:
                                legit_sub_edit = []
                                for sub_edit in not_necessarily_legit_edit:
                                    sub_s, sub_e = sub_edit
                                    already_done_string = "#" * (sub_e - sub_s + 1)
                                    if string[sub_s:sub_e] == already_done_string:
                                        continue
                                    else:
                                        legit_sub_edit.append(sub_edit)

                                if legit_sub_edit:
                                    potential_edits.append(tuple(legit_sub_edit))

                        for potential_edit in potential_edits:
                            # each potential edit is a tuple of 'ranges'
                            tmp_string = deepcopy(string)
                            full_length = len(tmp_string)
                            for edit in potential_edit:
                                # range (s, e)
                                s, e = edit
                                hash_replacement = "#"*(e-s+1)
                                tmp_string = [*tmp_string]
                                tmp_string[s:e+1] = [*hash_replacement]
                                tmp_string = "".join(tmp_string)
                            
                            is_ok = True
                            
                            assert full_length == len(tmp_string), (full_length, tmp_string)

                            if (max(rules)+1)*"#" not in tmp_string:
                                split_check_string = tmp_string.replace(".", "?")
                                tmp_splitted = [ughhhh for ughhhh in split_check_string.split("?") if ughhhh]
                                continuing_onwards = True
                                for tmp_split_val, rule in zip(tmp_splitted, rules):
                                    if not len(tmp_split_val) == rule:
                                        continuing_onwards = False
                                if continuing_onwards:
                                    for condition_to_satisfy in tmp_splitted:
                                        if (start_idx := tmp_string.find(condition_to_satisfy)) >= 0:
                                            tmp_string = tmp_string[0:start_idx] + tmp_string[start_idx + len(condition_to_satisfy):]
                                        else:
                                            is_ok = False
                                            break
                                    if is_ok:
                                        if "#" not in tmp_string:
                                            combinations_total += 1

                        
                        using_range = False
                        sub_rules = []
        
    return combinations_total

assert part1(example_lines) == 21
# part1(["?#?#?#?#?#?#?#? 1,3,1,6"])
# part1(["?###???????? 3,2,1"])
part1(input_lines)

AssertionError: more keys than rules errrr - ('???????.??.#?????', (2, 2))